In [1]:
import pandas as pd
import tensorflow as tf 
from sklearn import preprocessing
from tensorflow.python.client import device_lib
from keras.layers import Activation, Dense, Dropout, Input, Embedding, CuDNNLSTM, CuDNNGRU,  GlobalMaxPooling1D, GlobalAveragePooling1D,Reshape, Conv1D, MaxPooling1D
import sklearn.model_selection
from sklearn.model_selection  import train_test_split
import keras
import numpy as np 
import os
import keras
import pickle 

C:\Users\kin24\AppData\Local\conda\conda\envs\name_class\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kin24\AppData\Local\conda\conda\envs\name_class\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kin24\AppData\Local\conda\conda\envs\name_class\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\k

In [2]:
def re_balance(df, var):
    scale_factor = df['class'].value_counts().to_dict()

    min_v = np.max( list( scale_factor.values() ) )

    for k in scale_factor:
        scale_factor[k] = int( round( min_v/ scale_factor[k]) )


    return(pd.concat([
        pd.concat( [ df[df['class']==k] ] * scale_factor[k] ) for k in scale_factor
    ]))

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
print(device_lib.list_local_devices())

df = pd.read_parquet("./data/output/raw_names.parquet")
df = re_balance(df, 'class')

example = pd.DataFrame(data = 
                        {
                            'name' : ['CHAN TSZ KIN','CHEUNG KA SHING', 'YIU CHUNG YIM', 'LI SIJIE'], 
                            'class' : ["HK","HK","HK", "ML"]
                        })
example



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9310943868413481239
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11116255642
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11145140692909364944
physical_device_desc: "device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


,name,class
0,CHAN TSZ KIN,HK
1,CHEUNG KA SHING,HK
2,YIU CHUNG YIM,HK
3,LI SIJIE,ML


In [4]:
def get_tokenizer(x):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level = True, )
    tokenizer.fit_on_texts(x)
    tensor = tokenizer.texts_to_sequences(x)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    return tensor, tokenizer

x = df['name'].values
y = df['class'].values

tensor, tokenizer = get_tokenizer(x)

with open('./model/tokenizer.pkl' , 'wb') as f:
    pickle.dump(tokenizer, f)

le = preprocessing.LabelBinarizer()
le.fit(y)
label = le.transform(y)

with open('./model/le.pkl' , 'wb') as f:
    pickle.dump(le, f)


In [5]:
X_tmp, X_holdout, y_tmp, y_holdout = train_test_split(tensor, label, test_size=0.9)
X_train, X_tmp2, y_train, y_tmp2 = train_test_split(X_tmp, y_tmp, test_size=0.1)
X_val, X_test, y_val, y_test = train_test_split(X_tmp2, y_tmp2, test_size=0.5)

In [ ]:
max_char = np.max(tensor) +1 

input_x = Input(shape=(50,))
GRU_x = Embedding(max_char,30,input_length=50)(input_x)
GRU_x = CuDNNGRU(30, return_sequences=True)(GRU_x)
GRU_x = CuDNNGRU(30, return_sequences=True)(GRU_x)
GRU_x = CuDNNGRU(30, return_sequences=False)(GRU_x)
GRU_x = Activation('tanh')(GRU_x)
GRU_x = Dense(10, activation='sigmoid')(GRU_x)
GRU_x = Dropout(0.2)(GRU_x)
GRU_x = Dense(3, activation='sigmoid')(GRU_x)

GRU_model = keras.models.Model(inputs=[input_x], outputs=[GRU_x] )

GRU_model.compile(
    loss='categorical_crossentropy', 
    optimizer=keras.optimizers.adam(0.005),
    metrics=['accuracy'])

GRU_model.fit(
    X_train, 
    y_train, 
    batch_size=50, 
    epochs = 10,
    validation_data=(X_val, y_val) , 
    shuffle=True, 
    verbose = 1)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 112959 samples, validate on 6276 samples
Epoch 1/10
112959/112959 [==============================] - 26s 229us/step - loss: 0.6844 - acc: 0.6428 - val_loss: 0.4771 - val_acc: 0.7470
Epoch 2/10
112959/112959 [==============================] - 23s 203us/step - loss: 0.2277 - acc: 0.9044 - val_loss: 0.1087 - val_acc: 0.9630
Epoch 3/10
112959/112959 [==============================] - 22s 193us/step - loss: 0.0751 - acc: 0.9754 - val_loss: 0.0577 - val_acc: 0.9810
Epoch 4/10
112959/112959 [==============================] - 23s 200us/step - loss: 0.0483 - acc: 0.9837 - val_loss: 0.0397 - val_acc: 0.9898
Epoch 5/10
112959/112959 [==============================] - 22s 194us/step - loss: 0.0488 - acc: 0.9836 - val_loss: 0.0299 - val_acc: 0.9904
Epoch 6/10
112959/112959 [====

In [ ]:
GRU_model.evaluate(X_test, y_test, batch_size=5000)

In [ ]:
GRU_model.evaluate(X_train, y_train, batch_size=5000)

In [ ]:
GRU_model.evaluate(X_val, y_val, batch_size=5000)

In [ ]:
GRU_model.save("./model/m1.h5")

In [ ]:
import numpy as np
def proc_x(x, tokenizer):
    tensor = tokenizer.texts_to_sequences(x)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post',maxlen=50)
    return tensor


def predict_name(name, model, tokenizer,le, max_len=50):
    fit = le.classes_[np.argmax(GRU_model.predict( proc_x(name, tokenizer) ), axis=1)]
    return pd.DataFrame({'name':name, 'prediction':fit})


In [ ]:
predict_name(["SIN WAI KEUNG","LAU KAM LUNG"], GRU_model, tokenizer, le, max_len=50)

In [ ]:
predict_name(["CHEN MEI", "MAO YULIN","ZHANG QING", "ZHANG JIE"], GRU_model, tokenizer, le, max_len=50)

In [ ]:
predict_name(["abc LTD"], GRU_model, tokenizer, le, max_len=50)